<a href="https://colab.research.google.com/github/shininglight99/ellison-archive/blob/master/10Year_US_treasury_Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install QuantLib-Python 

     |████████████████████████████████| 18.8MB 1.4MB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from bs4 import BeautifulSoup
import QuantLib as ql

In [6]:
def GET_DATE():
  req = requests.get("https://www.marketwatch.com/investing/bond/tmubmusd10y?countrycode=bx")
  html = req.text
  soup = BeautifulSoup(html, 'html.parser')
  data = soup.find("span", class_="WSJBase--card__timestamp--2xDXNOQK")
  date = data.text
  date = datetime.datetime.strptime(date, "%m/%d/%y").date()
  return date

In [ ]:
import re
import requests
from datetime import datetime
import pandas as pd

def get_quote(reference_date):
    if type(reference_date) != datetime:
        reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    maturity = ["01m", "03m", "06m", "01y", "02y", "03y", "05y", "07y", "10y", "30y"]
    bond_table = pd.DataFrame(columns = ["Maturity Date","Price" , "Coupon Rate"], \
                             index = maturity)
    for mat in maturity:
        url = f"https://www.marketwatch.com/investing/bond/tmubmusd{mat}?countrycode=bx"
        data = requests.get(url)
        _, _, _, price, _, _, coupon_rate, maturity_date = \
                re.findall(r"<span class=\"primary \">(.*?)</span>\n", data.text)
        maturity_date = datetime.strptime(maturity_date, "%b %d, %Y")#\
                                                            #.strftime("%Y-%m-%d")
        if mat in ("01m", "03m", "06m", "01y"):
            numer, denom = price.split("/")
            discount_yield = int(numer) / float(denom)
            n = (maturity_date - reference_date).days
            price = 100 / (1 + n * discount_yield / 365)
        else:
            price1, fraction = price.split()[0], price.split()[1]
            numer, denom = fraction.split("/")[0], fraction.split("/")[1]
            price = int(price1) + int(numer) / float(denom)
        
        maturity_date = maturity_date.strftime("%Y-%m-%d")
        coupon_rate = float(coupon_rate[:-1]) / 100
        bond_table.loc[mat] = [maturity_date, price, coupon_rate]
    return bond_table

get_quote(datetime.today())

In [11]:
#Test
reference_date = GET_DATE
quote = get_quote(reference_date)
print(quote)

NameError: ignored